<a href="https://colab.research.google.com/github/Astra-Stark/AYNA_AI/blob/main/Ayna_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install --upgrade diffusers transformers accelerate safetensors


In [ ]:
wandb.init(project="polygon-color-unet")
wandb.watch(model, log='all')


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: astha27sharma (astha27sharma-oyo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
!pip install --quiet torchmetrics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.9/981.9 kB 24.4 MB/s eta 0:00:00


In [ ]:
!pip install torch torchvision pillow matplotlib




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import os, json

In [10]:
# For use in Google Colab to upload and extract a zip file

from google.colab import files
import zipfile
import os

# 1. Upload ZIP file via browser dialog
uploaded = files.upload()

# 2. Extract first uploaded file (assumes only one zip uploaded)
for fn in uploaded.keys():
    zip_path = fn
    break

# 3. Unzip in desired directory (./dataset here)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("dataset")

print('Extraction completed.')


Extraction completed.


In [11]:
class PolygonColorDataset(Dataset):
    def __init__(self, json_path, input_dir, output_dir, color_list, train=True, transform=None):
        with open(json_path, 'r') as f:
            self.data = json.load(f)
        self.input_dir = input_dir
        self.output_dir = output_dir
        self.colors = color_list
        if transform is not None:
            self.transform = transform
        else:
            self.transform = transforms.Compose([
                transforms.Resize((128,128)),
                transforms.RandomHorizontalFlip() if train else transforms.Lambda(lambda x: x),
                transforms.RandomVerticalFlip() if train else transforms.Lambda(lambda x: x),
                transforms.ColorJitter(brightness=0.2, contrast=0.2) if train else transforms.Lambda(lambda x: x),
                transforms.ToTensor()
            ])
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        entry = self.data[idx]
        inp = Image.open(os.path.join(self.input_dir, entry['input_polygon'])).convert('RGB')
        outp = Image.open(os.path.join(self.output_dir, entry['output_image'])).convert('RGB')
        color_idx = self.colors.index(entry['colour'])
        inp = self.transform(inp)
        outp = self.transform(outp)
        return inp, torch.tensor(color_idx, dtype=torch.long), outp


In [12]:
color_list = ["red", "green", "blue", "yellow", "cyan", "magenta", "orange", "purple"]  # adjust as needed

train_set = PolygonColorDataset(
    json_path='dataset/dataset/training/data.json',
    input_dir='dataset/dataset/training/inputs',
    output_dir='dataset/dataset/training/outputs',
    color_list=color_list,
    train=True
)
val_set = PolygonColorDataset(
    json_path='dataset/dataset/validation/data.json',
    input_dir='dataset/dataset/validation/inputs',
    output_dir='dataset/dataset/validation/outputs',
    color_list=color_list,
    train=False
)
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
val_loader = DataLoader(val_set, batch_size=16)


In [13]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.double_conv(x)

class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )
    def forward(self, x):
        return self.maxpool_conv(x)

class Up(nn.Module):
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)
    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2, diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class ConditionalUNet(nn.Module):
    def __init__(self, in_channels, n_colors, base_c=64, out_channels=3, emb_dim=32, bilinear=True):
        super().__init__()
        self.color_emb = nn.Embedding(n_colors, emb_dim)
        self.input_conv = DoubleConv(in_channels + emb_dim, base_c)
        self.down1 = Down(base_c, base_c * 2)
        self.down2 = Down(base_c * 2, base_c * 4)
        self.down3 = Down(base_c * 4, base_c * 8)
        self.down4 = Down(base_c * 8, base_c * 8)
        self.up1 = Up(base_c * 16, base_c * 4, bilinear)
        self.up2 = Up(base_c * 8, base_c * 2, bilinear)
        self.up3 = Up(base_c * 4, base_c, bilinear)
        self.up4 = Up(base_c * 2, base_c, bilinear)
        self.outc = nn.Conv2d(base_c, out_channels, 1)

    def forward(self, x, color_idx):
        batch_size, _, H, W = x.shape
        col_emb = self.color_emb(color_idx)   # [B, emb_dim]
        col_cond = col_emb.unsqueeze(2).unsqueeze(3).expand(-1, -1, H, W)
        x = torch.cat([x, col_cond], dim=1)
        x1 = self.input_conv(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        out = self.outc(x)
        return out


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ConditionalUNet(in_channels=3, n_colors=len(color_list)).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for x, color, y in train_loader:
        x, color, y = x.to(device), color.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x, color)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    val_loss = 0
    model.eval()
    with torch.no_grad():
        for x, color, y in val_loader:
            x, color, y = x.to(device), color.to(device), y.to(device)
            out = model(x, color)
            val_loss += criterion(out, y).item()
    print(f"Epoch {epoch+1}: Train Loss {total_loss/len(train_loader):.4f} | Val Loss {val_loss/len(val_loader):.4f}")
    torch.save(model.state_dict(), "best_conditional_unet.pth")


Epoch 1: Train Loss 0.7760 | Val Loss 0.8111
Epoch 2: Train Loss 0.5200 | Val Loss 3.6055
Epoch 3: Train Loss 0.3235 | Val Loss 4.3453
Epoch 4: Train Loss 0.2548 | Val Loss 34.0631
Epoch 5: Train Loss 0.1614 | Val Loss 9.7716
Epoch 6: Train Loss 0.1491 | Val Loss 9.1737
Epoch 7: Train Loss 0.1306 | Val Loss 57.1943
Epoch 8: Train Loss 0.1102 | Val Loss 52.8838
Epoch 9: Train Loss 0.0877 | Val Loss 0.3671
Epoch 10: Train Loss 0.0799 | Val Loss 0.2526
Epoch 11: Train Loss 0.0768 | Val Loss 0.0790
Epoch 12: Train Loss 0.0699 | Val Loss 0.0664
Epoch 13: Train Loss 0.0745 | Val Loss 0.0703
Epoch 14: Train Loss 0.0725 | Val Loss 0.0799
Epoch 15: Train Loss 0.0666 | Val Loss 0.0888
Epoch 16: Train Loss 0.0678 | Val Loss 0.0829


In [ ]:
# Load the trained model
model.load_state_dict(torch.load("best_conditional_unet.pth", map_location=device))
model.eval()

# Visualize a validation prediction
x, color, y = val_set[0]
x = x.unsqueeze(0).to(device)
color = color.unsqueeze(0).to(device)
with torch.no_grad():
    pred = model(x, color)
    pred_img = pred.squeeze().cpu().clamp(0,1).permute(1,2,0).numpy()
    inp_img = x.squeeze().cpu().permute(1,2,0).numpy()
    gt_img = y.permute(1,2,0).numpy()
    plt.figure(figsize=(8,3))
    plt.subplot(1,3,1); plt.imshow(inp_img); plt.title("Input")
    plt.subplot(1,3,2); plt.imshow(gt_img); plt.title("GT")
    plt.subplot(1,3,3); plt.imshow(pred_img); plt.title("Prediction")
    plt.show()
